<h2> Spectroscopic data reduction : spectra module

In [25]:
from pyvista import imred, tv, spectra
import pyvista.data
from importlib_resources import files
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

pyvista uses a display tool defined in the tv module. To use the interactive
display in a notebook, set the display to be an external display window, e.g. with 
<code>
%matplotlib qt
</code>
Instantiate a tv object, here we just call it t, but you could call it whatever you want!

In [2]:
%matplotlib qt
t=tv.TV()

The basic tool for basic image reduction is a Reducer object, defined in the imred module. Instantiate a reducer here. The main argument is an instrument name, which tells it to read a YAML configuration file for the specified instrument. We also give it an optional dir= argument to specify the default directory from which to read images, if a directory is not specified in subsequent commands that read images.

In [26]:
#indir='/home/users/adijeau/Documents/UT220107'
indir='/home/holtz/red/UT211030/kosmos/'
red=imred.Reducer('KOSMOS',dir=indir)

INSTRUMENT: KOSMOS   config: 
  will use format:  /home/holtz/red/UT211030/kosmos//*{:04d}.f*.fits*
         gain:  [0.6]    rn: [ 5.]
         scale:  None   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  2055    43    20  4056 
  2105    43    20  4056 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  4096 
     0  2048     0  4096 
  Norm box: 
    SC    NC    SR    NR
  1000    51  2000    51 


In [27]:
red.log().show_in_notebook()


file /home/holtz/red/UT211030/kosmos/BD28d4211.0009.fits does not have DATE-OBS


idx,FILE,DATE-OBS,OBJNAME,RA,DEC,EXPTIME
0,BD28d4211.0009.fits,,,,,
1,flat.0001.fits,2021-10-30T00:24:54.220537,,7:28:00.00,80:00:00.00,5.0
2,flat.0002.fits,2021-10-30T00:27:30.532534,,7:28:00.00,80:00:00.00,20.0
3,flat.0003.fits,2021-10-30T00:28:53.571555,,7:28:00.00,80:00:00.00,5.0
4,flat.0004.fits,2021-10-30T00:31:04.210227,,7:28:00.00,80:00:00.00,20.0
5,flat.0005.fits,2021-10-30T00:31:59.111371,,7:28:00.00,80:00:00.00,20.0
6,flat.0006.fits,2021-10-30T00:32:54.113381,,7:28:00.00,80:00:00.00,20.0
7,HeNeAr.0007.fits,2021-10-30T00:36:06.854869,,7:28:00.00,80:00:00.00,60.0
8,HeNeAr.0008.fits,2021-10-30T00:39:12.597137,,7:28:00.00,80:00:00.00,180.0
9,BD28d4211.0010.fits,2021-10-30T01:08:22.357120,BD+28d4211,21:51:11.07,28:51:51.80,60.0


A main method of the reducer object is the reduce() method. Without any additional arguments, reduce() will read an image from disk, subtract the overscan (region(s) as determined from the instrument configuration file), compute an uncertainty array using the gain and readout noise from the instrument configuration file, and return a CCDData object with the data, uncertainty, and mask. 
<p>
To specify the input image, we could pass a string with the file name. If the string does not include a '/', it will read from the default input directory.
<p>
If the file can be identified with a unique integer, then you can just specify this number, which can be very convenient. This is turned into a character string using the formstr attribute define in the configuration file, which is used to search for the file to read.
<p>
We can display the image using the tv() method of our display tool, which can take as input a Data object, a numpy array, or a FITS HDU object.

In [28]:
a=red.reduce(12)
t.tv(a)

  Reading file: /home/holtz/red/UT211030/kosmos/GP013_parad.0012.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


The resulting Data object has attributes: header, data, uncertainty, and bitmask, which are carried along through the processing.

In [29]:
a.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                   32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 2148                                                  
NAXIS2  =                 4096                                                  
EXTEND  =                    T                                                  
OBSERVAT= 'APO'                / Per the IRAF observatory list.                 
TELESCOP= '3.5m'                                                                
INSTRUME= 'kosmos'             / Instrument name                                
LATITUDE= +3.2780361000000E+01 / Latitude of telescope base                     
LONGITUD= -1.0582041700000E+02 / Longitude of telescope base                    
UTC-TAI = -37.0                / UTC = TAI + UTC_TAI(seconds)                   
UT1-TAI = -37.10            

<h4> Calibration: make and apply flat field

If we add additional arguments to reduce(), we can add additional calibration steps. For example, to flat field the data, we would add a flat= keyword through which we give the reducer a flat field.
<br>
First, however, we have to make the flat field, which is accomplished using the mkflat() method, which takes as input a list of frames to be used to construct the master flat field (superflat). For a spectrograph, we use the spec=True keyword which will remove the spectral signature from the combined flat.
<br>
If you specify a pyvista TV tool with the display= keyword, mkflat() will display the final flat field, then each individual input frame divided by
the final flat, so you can inspect and confirm that all input frames are similar. You will need to enter a space in the display window after each
image is displayed to proceed to the next, as prompted by the output.

In [116]:
flatims=[18,19,20,21,22]
flat=red.mkflat(flatims,spec=True,display=None,littrow=False)

  Reading file: /home/holtz/red/UT211030/kosmos/flat.0018.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/flat.0019.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/flat.0020.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....
  See final image, use - key for S/N image.
   To continue, hit space in display window (p for debug) 
    see image: flat.0018.fits divided by master
   To continue, hit space in display window (p for debug) 
    see image: flat.0019.fits divided by master
   To continue, hit space in display window (p for debug) 
    see image: flat.0020.fits divided by master
   To continue, hit space in display window (p for debug) 


In [31]:
t.tv(flat,min=0.5,max=1.5)

Read, overscan subtract, flat field, and display a star spectral image

In [32]:
star=red.reduce(10,flat=flat)
t.tv(star,max=1000) 

  Reading file: /home/holtz/red/UT211030/kosmos/BD28d4211.0010.fits
  subtracting overscan vector 
  subtracting overscan vector 
  flat fielding...
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Here is an example of attempting cosmic ray rejection. CR rejection can be challenging; the main concern is that the cosmic ray rejection flags object pixels as cosmic rays! Users should be very careful about this. The Reducer object provides two algorithms for CR-rejection. If crbox='lacosmic", then the LA Cosmic routine of van Dokkum, as implemented
in the astroscrappy package, is used. If crbox=[nrow,ncol], then the image
is median-filtered with the specfied box shape, and pixels above nsig (default=5) times the uncertainty are replaced with the median; if multiple values of crsig are input as a list (e.g., crsig=[5,2], then multiple iterations are done, where iterations after the first only apply to neighbors of CRs identified in previous iterations. For spectroscopy, beware: a box aligned with the wavelength direction will avoid flagging pixels in the object, but may flag pixels in sky lines, while the reverse is true for a box aligned along the slit.

Note that you can include cosmic ray rejection in the reduce() method by adding a crbox= (and, optionally, a crsig=) keyword.

In [33]:
crstar=red.crrej(star,crbox=[9,1],crsig=[5,2],display=t)

  Iteration 0, zapping CRs with filter [9,1]...
  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 
  Iteration 1, zapping CRs with filter [9,1]...
  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 


<h4> Tracing and extraction

We want to extract the spectrum, i.e. from the 2D image to a 1D spectrum. Start with a trace previously determined for KOSMOS. The Trace object sets the order of the polynomial used to fit the trace (degree attribute), the length of the slit (rows attribute), the range of pixels along the slit to be used to automatically find an object, etc.

In [42]:
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
vars(trace) 

[autoreload of pyvista.spectra failed: Traceback (most recent call last):
  File "/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/holtz/git/pyvista/python/pyvista/spectra.py", line 1228
    print('ap

{'type': 'Polynomial1D',
 'degree': 2,
 'sc0': 2048,
 'pix0': 0,
 'spectrum': array([    0.,     0.,     0., ...,  3448.,  3443.,  3442.]),
 'rad': 5,
 'lags': array([-300, -299, -298, -297, -296, -295, -294, -293, -292, -291, -290,
        -289, -288, -287, -286, -285, -284, -283, -282, -281, -280, -279,
        -278, -277, -276, -275, -274, -273, -272, -271, -270, -269, -268,
        -267, -266, -265, -264, -263, -262, -261, -260, -259, -258, -257,
        -256, -255, -254, -253, -252, -251, -250, -249, -248, -247, -246,
        -245, -244, -243, -242, -241, -240, -239, -238, -237, -236, -235,
        -234, -233, -232, -231, -230, -229, -228, -227, -226, -225, -224,
        -223, -222, -221, -220, -219, -218, -217, -216, -215, -214, -213,
        -212, -211, -210, -209, -208, -207, -206, -205, -204, -203, -202,
        -201, -200, -199, -198, -197, -196, -195, -194, -193, -192, -191,
        -190, -189, -188, -187, -186, -185, -184, -183, -182, -181, -180,
        -179, -178, -177, -

Alternatively, you can just instantiate a fresh Trace object, specifying whether your input images have wavelength along columns (transpose=False) or rows (transpose=True), and specifying the extent of the slit with the rows= keyword. Other keywords allow you to specify other attributes, see the help.

In [43]:
mytrace=spectra.Trace(transpose=True,rows=[550,1450])
vars(mytrace)

{'type': 'Polynomial1D',
 'degree': 2,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-50, 50),
 'rows': [550, 1450],
 'model': None,
 'sigmodel': None}

We can use the existing trace to retrace a new spectrum. This is done by cross-correlating the reference spectrum with a spatial cut across the input spectrum (at sc0) to find the shift, then using the old model to start the trace from that starting position. The functional form of the old model is preserved:

In [44]:
trace.retrace(star,display=t)

  Derived pixel shift from input trace:  0.0254599566667
Using shift:  0.0254599566667


/home/holtz/git/pyvista/python/pyvista/spectra.py:1111: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
/home/holtz/git/pyvista/python/pyvista/spectra.py:1122: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<rad))[0]



  See trace. Hit space bar to continue....


Here are some various alternatives for finding traces:

A. one could use the find() method to determine the shift between the stored trace and your frame automatically with cross correlation, then just go straight to extraction using the shape of the stored trace; this would be applicable if your object is too faint, or doesn't have sufficient continuum, to trace. If you want to mark the location of the object manually, use the inter=True keyword (along with display=) in find(). 


In [45]:
trace.model[0](2048)
trace.find(star,inter=True,display=t)
print(trace.pix0)

Hit "f" on location of spectrum: 
setting trace offset to:  -11.3964816473
-11.3964816473


B. You can just find peak(s) in the spatial direction using findpeak(), then trace them:

In [49]:
peaks,fibers=trace.findpeak(star,thresh=50)
                            
trace.trace(star,peaks,display=t)

looking for peaks using 200 pixels around 2048, threshhold of 50.000000
peaks:  [394]
aperture/fiber:  [0]


/home/holtz/git/pyvista/python/pyvista/spectra.py:1111: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
/home/holtz/git/pyvista/python/pyvista/spectra.py:1122: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<rad))[0]



  See trace. Hit space bar to continue....


The model is now modified for the input spectrum:

In [51]:
trace.model[0](2048)

943.8185449234705

C. If you needed to create Trace from scratch, instantiate one with rows=, lags=, sc= (transpose= determines whether input image needs to be transposed to get wavelength changing along columns). 
<p>
    Note the skip= keyword which can be used when tracing to speed things up by only computing centroids every skip pixels, taking a median around these pixels of width skip pixels. The default is skip=10

In [53]:
trace=spectra.Trace(sc0=2048,lags=range(-300,300),
                    rows=[550,1450],transpose=red.transpose)
vars(trace)
srow=[955]   # list to allow for multiple spectra on an image, mnaually set
srow,fibers=trace.findpeak(star, thresh=50)  # alternatively, find peak(s)
# trace.find(star) will find the highest peak by cross-correlation
# trace.find(star,inter=True,display=t)  will let you mark a trace location
starec=trace.trace(star,srow,display=t,skip=100) 
vars(trace)

looking for peaks using 200 pixels around 2048, threshhold of 50.000000
peaks:  [394]
aperture/fiber:  [0]


/home/holtz/git/pyvista/python/pyvista/spectra.py:1111: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
/home/holtz/git/pyvista/python/pyvista/spectra.py:1122: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<rad))[0]



  See trace. Hit space bar to continue....


{'type': 'Polynomial1D',
 'degree': 2,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': array([ 54208.6171875 ,   8355.84960938,      0.        , ...,
         10526.25      ,  22835.43359375,  14175.71191406]),
 'rad': 5,
 'transpose': True,
 'lags': range(-300, 300),
 'rows': [550, 1450],
 'model': [<Polynomial1D(2, c0=936.34518165949, c1=0.0034208334284421887, c2=1.1122901421158688e-07)>],
 'sigmodel': None,
 'sc0': 2048,
 'index': array([0])}

Extraction is done using the extract() method of the Trace object, currently just with simple boxcar extraction. Use rad= to specify window radius. Use back=[[b1,b2],[b3,b4]] to subtract background as determined from one or more background windows (note argument should be list of 2-element lists, giving start and end pixel, relative to object location, of each background window).

In [54]:
starec=trace.extract(star,display=t,rad=25)
print(starec.shape)

  extracting ... (may take some time,
                   consider threads= if multithreading is available
  See extraction window(s). Hit space bar to continue....

(1, 4096)


extract() returns a pyvista Data object, with extracted spectrum, and uncertainty. Note that this is a 2D array to accomodate multiple objects/traces, even if there is only a single object.

In [55]:
plt.figure()
plt.plot(starec.data[0])
plt.plot(starec.data[0]/starec.uncertainty.array[0])

Background can be subtracted during extraction using the back= keyword, which takes a list of 2-element lists, given the relative location, in pixels, of the background window(s) (relative to the location of the trace):

In [56]:
starec2=trace.extract(star,rad=25,back=[[50,75],[-75,-50]],display=t)
plt.figure()
plt.plot(starec.data[0])
plt.plot(starec2.data[0])
plt.show()

  extracting ... (may take some time,
                   consider threads= if multithreading is available
  See extraction window(s). Hit space bar to continue....



<h4> Wavelength calibration

Now let's turn to wavelength calibration, i.e. getting a function that gives the wavelength as a function of pixel. We'll solve for this using arc frames, here taken with each lamp separately, so sum the three exposures

In [57]:
#Frame 15 is He, 16 is Ne, and 17 is Ar
arcs=red.sum([15,16,17])
t.clear()
t.tv(arcs)

  Reading file: /home/holtz/red/UT211030/kosmos/He.0015.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  Reading file: /home/holtz/red/UT211030/kosmos/Ne.0016.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  Reading file: /home/holtz/red/UT211030/kosmos/Ar.0017.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  combining data with sum....
  calculating uncertainty....


In [58]:
arcec=trace.extract(arcs,display=None,rad=20)

  extracting ... (may take some time,
                   consider threads= if multithreading is available



Wavelength calibration first stars with identifying lines. This is much easier if one can work from a previous solution. Here we start by reading a previous solution into a pyvista WaveCal object.

For KOSMOS, there are previous solutions for low, central, and high slit positions

In [68]:
spectra.WaveCal("?")

available predefined WaveCals: 
ARCES/ARCES_wave.fits
BOSS/BOSS_b1_waves.fits
BOSS/BOSS_b2_waves.fits
BOSS/BOSS_r1_waves.fits
BOSS/BOSS_r2_waves.fits
DIS/DIS_blue_waves.fits
DIS/DIS_red_waves.fits
KOSMOS/KOSMOS_blue_cent_waves.fits
KOSMOS/KOSMOS_blue_high_waves.fits
KOSMOS/KOSMOS_blue_low_waves.fits
KOSMOS/KOSMOS_blue_waves.fits
KOSMOS/KOSMOS_red_high_waves.fits
KOSMOS/KOSMOS_red_low_waves.fits
KOSMOS/KOSMOS_red_waves.fits


In [63]:
wav=spectra.WaveCal('KOSMOS/KOSMOS_red_low_waves.fits')
vars(wav)

  rms:    0.230 Angstroms (60 lines)


{'type': 'chebyshev',
 'degree': 3,
 'ydegree': 2,
 'waves': array([ 5400.562  ,  5562.22534,  5570.28944,  5572.548  ,  5580.38729,
         5748.299  ,  5764.418  ,  5824.5185 ,  5852.4878 ,  5866.75017,
         5870.91599,  5875.618  ,  5879.90035,  5944.8342 ,  5945.45239,
         5977.64573,  5993.8502 ,  6029.9971 ,  6074.3377 ,  6096.163  ,
         6143.0623 ,  6163.5939 ,  6217.2813 ,  6266.495  ,  6304.7892 ,
         6334.4279 ,  6382.9914 ,  6402.246  ,  6506.5279 ,  6532.8824 ,
         6598.9529 ,  6678.2    ,  6717.0428 ,  6752.832  ,  6871.29   ,
         6965.43   ,  7032.4127 ,  7065.188  ,  7147.041  ,  7173.939  ,
         7245.167  ,  7383.98   ,  7438.899  ,  7635.105  ,  7723.8    ,
         7891.075  ,  7948.175  ,  8082.458  ,  8264.521  ,  8408.208  ,
         8424.647  ,  8521.441  ,  8620.4602 ,  8634.647  ,  8654.3831 ,
         8667.9442 ,  8697.94   ,  8799.0875 ,  8853.866  ,  8919.5007 ]),
 'waves_order': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [64]:
wav.dispersion()

-1.0184165540022461

With a previous solution loaded, the identify routine will cross correlate the input spectrum with the previous solution, then attempt to identify the lines from the previous solution at the shifted pixel position from the previous solution. Finally, it does a fit.

The identify() method fits Gaussians to the lines to get their position, and gets FWHM in the process. rad= keyword specifies number of pixels on either side of line to use in fit. If you add the file= keyword, then the routine will try to identify all of the lines from the reference file; this might be useful if you wanted to extend the wavelength range of the lines from the initial object, e.g. if you were working at a different grating tilt or slit location. If you did that, you might want to save your WaveCal object after cleaning the line list for good lines, so you could use that as a starting guess for other data taken at a similar wavelength setting.

In [67]:
wav.identify(arcec,plot=True,rad=10) #,file='henearkr.dat')

  cross correlating with reference spectrum using lags:  range(-300, 300)
  Derived pixel shift from input wcal:  [ -6.86384283e-09]
  See identified lines.
  rms:    0.130 Angstroms (44 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       anything else : finish and return
  rms:    0.130 Anstroms
  input from plot window...
  rms:    0.108 Anstroms
  input from plot window...



Note that once you remove lines, they are removed for subsequent uses of the WaveCal object.

If you want to refit with existing line positions, e.g. to try a different model, you can just call fit().

In [69]:
wav.fit(degree=5)  

  rms:    0.070 Angstroms (44 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       anything else : finish and return
  rms:    0.070 Anstroms
  input from plot window...


KeyboardInterrupt: 

Show the FWHM of the lines as a function of wavelength

In [70]:
wav.fwhm
t.plotax2.cla()
gd=np.where(wav.weights > 0)[0]
t.plotax2.plot(wav.waves[gd],np.abs(wav.fwhm[gd]),'ro')

OK, now use the wavelength solution to get wavelength as a function of pixel. We can save that in the wave attribute of our Data object.

In [71]:
wav.add_wave(starec)
plt.figure()
plt.plot(starec.wave[0],starec.data[0])

Here is a way to get the inverse wavelength solution, i.e. pixels as a function of wavelength, using a cubic spline fit to the wavelength solution.

In [72]:
# get inverse relation, i.e. pixels as f(wavelength)
pix=np.arange(4096)
from scipy.interpolate import CubicSpline
wav2pix=CubicSpline(np.flip(starec.wave[0]),np.flip(pix))
print('wavelength 6563 expected at pixel: ',wav2pix(6463))

wavelength 6563 expected at pixel:  2462.3885165225547


Next cell demonstrates resampling of spectrum onto a uniform wavelength grid. This may not be desired, as it leads to correlated uncertainties between pixels! But it would be needed if, e.g., you wanted to cross-correlate spectra to find velocities, in which case you would want a spectrum with constant dispersion in log(lambda).

In [73]:
plt.figure()
plt.clf()
# original data with wavelength solution
plt.plot(starec.wave[0],starec.data[0])

# set desired new wavelength scale:
wnew=10**np.arange(3.5,4.0,5.5e-6)
# resampled data on constant log(lambda) grid
plt.plot(wnew,wav.scomb(starec,wnew).data)

/home/holtz/git/pyvista/python/pyvista/spectra.py:757: RuntimeWarning: invalid value encountered in true_divide
  out = out / sig
/home/holtz/git/pyvista/python/pyvista/spectra.py:758: RuntimeWarning: divide by zero encountered in true_divide
  sig = np.sqrt(1./sig)


<h4> Adjusting wavelength solution for flexure

In [74]:
objec=trace.extract(red.reduce(12),display=t,rad=20)
wav.add_wave(objec)

  Reading file: /home/holtz/red/UT211030/kosmos/GP013_parad.0012.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  extracting ... (may take some time,
                   consider threads= if multithreading is available
  See extraction window(s). Hit space bar to continue....



In [76]:
plt.figure()
plt.plot(objec.wave[0],objec.data[0])

The skyline() method will addjust the current wavelength solution based on finding sky emission lines. The default behavior is to adopt the current wavelength solution but allow only the 0th order term to shift. If you specify linear=True, then the 1st order term is also allowed to vary; beware that you must have multiple sky lines for this to be effective! Otherwise, skyline() takes most of the same keywords as identify()
<br>
Here, I first copy the WaveCal object so we can compare the original to the corrected solution.

In [77]:
import copy
swav=copy.deepcopy(wav)
swav.skyline( objec, thresh=50)

  See identified lines.
  rms:    2.220 Angstroms (17 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       anything else : finish and return
  rms:    2.220 Anstroms
  input from plot window...
  rms:    0.858 Anstroms
  input from plot window...
  rms:    0.625 Anstroms
  input from plot window...



See how the model has been modified compared to the original:

In [78]:
print(wav.model)
print(swav.model)

Model: Chebyshev1D
Inputs: ('x',)
Outputs: ('y',)
Model set size: 1
Degree: 5
Domain: (726.33296998475009, 4139.4718565617895)
Parameters:
          c0            c1             c2      ...        c4             c5      
    ------------- -------------- ------------- ... --------------- --------------
    7094.14264822 -1738.60683552 33.0204776727 ... 0.0297016859193 0.119578017836
Model: Chebyshev1D
Inputs: ('x',)
Outputs: ('y',)
Model set size: 1
Degree: 5
Domain: (726.33296998475009, 4139.4718565617895)
Parameters:
         c0           c1             c2      ...        c4             c5      
    ----------- -------------- ------------- ... --------------- --------------
    7093.961185 -1738.60683552 33.0204776727 ... 0.0297016859193 0.119578017836


<h4> Flux calibration

Instantiate a FluxCal object. You can set it up to use a polynomial fit to the response curve by specifying the polynomial degree with the degree= keyword. If you set degree=-1, the response curve will be determined by a median of the individual response curves, optionally smoothed with a median filter over wavelength (see response() method below).

In [79]:
flx=spectra.FluxCal(degree=-1)
polyflx=spectra.FluxCal(degree=5)

Load in several extracted spectra of flux standards, here all of BD+28 4211. The standard star spectrum is given using the file= keyword, where the input file should have labelled columns wave, flux, and bin. Alternatively, you can pass an astropy Table with (at least) these three columns, using the stdflux= keyword

In [80]:
for im in [10,11] :
    star=red.reduce(im)
    trace.retrace(star)
    starec=trace.extract(star)
    wav.add_wave(starec)
    flx.addstar(starec[0],starec.wave[0],file='flux/okestan/fbd28d4211.dat')
    polyflx.addstar(starec[0],starec.wave[0],file='flux/okestan/fbd28d4211.dat')

  Reading file: /home/holtz/red/UT211030/kosmos/BD28d4211.0010.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  Derived pixel shift from input trace:  -0.05082672912
Using shift:  -0.05082672912


/home/holtz/git/pyvista/python/pyvista/spectra.py:1111: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
/home/holtz/git/pyvista/python/pyvista/spectra.py:1122: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<rad))[0]



  extracting ... (may take some time,
                   consider threads= if multithreading is available

  Reading file: /home/holtz/red/UT211030/kosmos/BD28d4211.0011.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  Derived pixel shift from input trace:  -0.21185874283
Using shift:  -0.21185874283
  Tracing row: 943
  extracting ... (may take some time,
                   consider threads= if multithreading is available



Solve for the median response curve, taking a median filter over this median.

In [81]:
flx.response(plot=True,medfilt=200)
polyflx.response(plot=True)

Apply the response curve to an object. Note that, using Data objects, the S/N is preserved!

In [82]:
fig,ax=plots.multi(1,2,hspace=0.001,sharex=True)
ax[0].plot(starec.wave[0],starec.data[0])
ax[1].plot(starec.wave[0],starec.data[0]/starec.uncertainty.array[0])
flx.correct(starec,starec.wave)
ax[0].plot(starec.wave[0],starec.data[0])
ax[1].plot(starec.wave[0],starec.data[0]/starec.uncertainty.array[0])

<h3> longslit extraction and wavelength calibration

For extended objects, and perhaps for sky subtraction, we might want to work along the slit. The wavelength solution varies along the slit (line curvature), usually with more than just an offset.

Start by working along the slit to identify lines

In [83]:
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
arc2d=trace.extract2d(arcs)
t=tv.TV()
t.tv(arc2d)

In [84]:
from pyvista import image
wav=spectra.WaveCal(file='KOSMOS/KOSMOS_red_low_waves.fits')
image.smooth(arc2d,[5,1])
t.clear()
t.tv(arc2d)
wav.identify(arc2d, rad=10,display=None, plot=True,
              nskip=20,lags=np.arange(-10,10))
wav.add_wave(arc2d)

  rms:    0.230 Angstroms (60 lines)
  cross correlating with reference spectrum using lags:  [-10  -9  -8  -7  -6  -5  -4  -3  -2  -1   0   1   2   3   4   5   6   7
   8   9]
  Derived pixel shift from input wcal for row: 899 7
  See identified lines.
  rms:    2.606
rejecting 57 points from 1709 total: 
  rms:    0.820
rejecting 57 points from 1709 total: 
  See 2D wavecal fit. Enter space in plot window to continue



In [85]:
wav.plot()


/home/holtz/git/pyvista/python/pyvista/spectra.py:260: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  self.fig.tight_layout()


OK, use the solution to make a wavelength map

In [86]:
t.clear()
t.tv(arc2d.wave)

Subtract out the central wavelength solution to see how the solution varies with row

In [87]:
dw=arc2d.wave-arc2d.wave[450]
t.tv(dw)  

Now do a 2d extraction of an object, and attach the wavelengths

In [105]:
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
obj=red.reduce(12,crbox=[1,9],crsig=[5,2])
t.tv(obj)
obj2d=trace.extract2d(obj)
wav.add_wave(obj2d)
t.tv(obj2d)


  Reading file: /home/holtz/red/UT211030/kosmos/GP013_parad.0012.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  Iteration 0, zapping CRs with filter [1,9]...
  Iteration 1, zapping CRs with filter [1,9]...


In [98]:
t.tv(obj.bitmask)
t.tv(obj2d.bitmask)

In [97]:
star=red.reduce(10)
star2d=trace.extract2d(star)
wav.add_wave(star2d)

  Reading file: /home/holtz/red/UT211030/kosmos/BD28d4211.0010.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Here we rectify the image to have a constant wavelength scale (in log lambda). We choose the new scale based on the starting and ending wavelengths in the original image, and resample to get the same number of pixels.

If you preferred not to resample your object (or at least minimize the ressampling), you could resample the image to the wavelength array at the location of your object

In [99]:
wlim=(arc2d.wave[450,0],arc2d.wave[450,-1])
wnew=10.**np.linspace(np.log10(wlim[1]),np.log10(wlim[0]),4095)
print(wnew)
arc2d_rect=wav.correct(arc2d,wnew)
t.tv(arc2d)
t.tv(arc2d_rect)

[ 4962.95599479  4963.68174822  4964.40760777 ...,  9028.15237881
  9029.47260259  9030.79301943]


In [106]:
t.tv(obj2d)
obj2d_rect=wav.correct(obj2d,wnew)
star2d_rect=wav.correct(star2d,wnew)
t.tv(obj2d_rect)

Note that the bitmask is propagated to the rectified image, where any output pixel that has a contribution from an input pixel with a bit set, has that same bit set in the output.

In [107]:
t.tv(obj2d_rect.bitmask)

OK, now we can extract in the wavelength rectified image for better sky subtraction. Create a new trace object with the rows= attribute to give the new extent of the image in rows, since our previous extraction limited the extraction to the length of the slit, as set by the rows attribute in the original trace object.

In [113]:
rows=[0,star2d_rect.shape[0]]
trace=spectra.Trace(sc0=int(star2d_rect.shape[1]/2.),rows=rows)
t.clear()
rows,fibers=trace.findpeak(star2d_rect,thresh=20)
print('rows: ',rows)
trace.trace(star2d_rect,rows,display=t)
ext=trace.extract(obj2d_rect,display=t,rad=10)
ext.add_wave(wnew)
ext_sub=trace.extract(obj2d_rect,rad=10,back=[[20,30]],display=t)
ext_sub.add_wave(wnew)

looking for peaks using 200 pixels around 2047, threshhold of 20.000000
peaks:  [363, 394, 426]
aperture/fiber:  [0, 1, 2]
rows:  [363 394 426]
self.sc0:  2047


/home/holtz/git/pyvista/python/pyvista/spectra.py:1112: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:1123: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:


/home/holtz/git/pyvista/python/pyvista/spectra.py:1112: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:1123: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:


/home/holtz/git/pyvista/python/pyvista/spectra.py:1112: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:1123: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:



  See trace. Hit space bar to continue....
  extracting ... (may take some time,
                   consider threads= if multithreading is available
  See extraction window(s). Hit space bar to continue....

  extracting ... (may take some time,
                   consider threads= if multithreading is available
  See extraction window(s). Hit space bar to continue....



In [114]:
plt.figure()
plt.plot(ext.wave,ext.data[0])
plt.plot(ext.wave,ext_sub.data[0])

CR rejection via multiple image stacking

<h4> KOSMOS blue channel

In [ ]:
bwav=spectra.WaveCal('KOSMOS/KOSMOS_blue_waves.fits')
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
arcs=red.sum([49,50,51])
arcec=trace.extract(arcs)
bwav.identify(arcec,plot=True)


In [ ]:
objec=trace.extract(red.reduce(26),display=None,rad=20)
bwav.add_wave(objec)
plt.figure()
plt.plot(objec.wave[0],objec.data[0])

swav=copy.copy(bwav)
swav.model.fixed['c0']=False
swav.model.fixed['c1']=True
swav.model.fixed['c2']=True
swav.model.fixed['c3']=True 
swav.identify(objec,wav=objec.wave,file='skyline.dat',plot=True,thresh=50,inter=True)
print(bwav.model)
print(swav.model)

In [ ]:
flx=spectra.FluxCal(degree=5)
for im in [34,35,36] :
    star=red.reduce(im)
    trace.retrace(star)
    starec=trace.extract(star)
    bwav.add_wave(starec)
    flx.addstar(starec[0],wave[0],file='flux/okestan/ffeige34.dat')

In [ ]:
flx.response(plot=True)

In [ ]:
plt.figure()
flx.correct(starec,starec.wave[0])
plt.plot(starec.wave[0],starec.data[0])